# A Demonstration of the Diffie&ndash;Hellman Key Agreement Protocol
Below I illustrate a protocol that permits two parties (Alice and Bob) to exchange a confidential message on an insecure channel, and which prevents a third party (Eve) who intercepts the message from learning its content.

Whitfield Diffie and Martin Hellman first introduced this protocol in their seminal 1976 paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf). This protocol solved a problem that had afflicted all previously known ciphers; that of exchanging secret encryption keys *securely* between trusted parties as a prerequisite to private communication. The so&ndash;called *Diffie&ndash;Hellman key agreement protocol* remains to this day the de facto method for exchanging encryption keys on insecure channels.

In the same paper, Diffie and Hellman further imagined a novel cryptosystem in which a publicly&ndash;known key could be used to encrypt a message that only the owner of a different, *secret* key could decrypt. Such a method&mdash;what they termed a *public&ndash;key* cryptosystem&mdash;could further be employed to guarantee the authenticity of a message (a property that encryption alone could not provide), via a process they called *digital signature*.

Although Diffie and Hellman failed to realize a practical implementation for such a public&ndash;key cryptosystem, they did succeed in laying the essential groundwork for it (it would fall to Ronald Rivest, Adi Shamir and Leonard Adlemen to flesh it out in their 1978 paper [*A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems*](https://people.csail.mit.edu/rivest/Rsapaper.pdf)). However, Diffie and Hellman did leave us with a practical implementation for secure key&ndash;exchange; one that remains to this day the de facto method for exchanging single (or *symmetric*) keys over insecure channels. The method is based on a [one&ndash;way function](https://en.wikipedia.org/wiki/One-way_function) that pits the difficulty of solving the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) (or DLP) in large&ndash;order [multiplicative integer groups](https://en.wikipedia.org/wiki/Multiplicative_group) against the ease of multiplication in such groups.

## The Source Code
The following sequence of function calls to the [dh](https://github.com/dchampion/crypto/tree/master/src/core/dh.py) module illustrates a session in which Alice encrypts and transmits a  message to Bob over an insecure channel, which Bob successfully decrypts upon receipt. Meanwhile, an adversary (Eve) intercepts the message but, because the message is encrypted, has no feasible way of learning its content.

It is assumed that Alice, Bob and Eve all possess their own copy of the [dh](https://github.com/dchampion/crypto/tree/master/src/core/dh.py) module; but that Alice and Bob keep the keys generated by this module secret.

At the top of each code snippet is a line&ndash;comment to indicate who (i.e., Alice, Bob and/or Eve) is invoking the code. Illustrative code snippets, i.e., those that are not part of the protocol, but rather merely informative, are not commented.

In [1]:
import os
os.chdir("../src")

# Alice and Bob (and possibly Eve)
from core import dh
from tests.core import sym

Alice and Bob both import the `dh` module, and a module `sym` that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the `sym` cipher used here is for illustrative purposes only, and in a real-world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as 3DES or AES).

In [2]:
# Alice
q, p, g = dh.generate_parameters(2048)

Alice begins by computing the domain parameters she and Bob will use to generate their keys.

The argument passed to the function `dh.generate_parameters` specifies the size, in bits, of the modulus to be used in the protocol; the security of the system is directly proportional to the size of this modulus.

The parameter *p* returned by `dh.generate_parameters` is the prime modulus. The parameter *q*, a 256&ndash;bit prime number, is the size of the smallest subgroup of the full group modulo *p*; all keys generated using these parameters will be confined to this subgroup. Finally, the parameter *g* is the generator of this subgroup.

All three parameters returned by this function are public, and may be observed by Eve with no compormise to the security of the system.

In [3]:
print(p.bit_length())
print(q.bit_length())

2048
256


Here we print the bit lengths of the first two parameters returned by `dh.generate_parameters`. Note that the length of *p*, the  modulus, is in fact the 2048 bits requested (or, due to the implementation details of the protocol, 2047 bits). The length of *q*, 256 bits, represents the size of smallest subgroup of the full group modulo *p*.

We are not particularly interested in the bit&ndash;length of the generator *g*, but rather that it generates the entire subgroup of order *q*. Alice verifies this by calling the function `dh.validate_parameters`.

In [4]:
# Alice
dh.validate_parameters(q, p, g)

If `dh.validate_parameters` returns without raising an exception, Alice knows the parameters are suitable to generate keys to be used for a secure message exchange.

In [5]:
# Alice
kA, KA = dh.generate_keypair(q, p, g)

Next, Alice computes a pair of keys; *kA* is her private key, and *KA* her public key (the keys are suffixed with the captital letter *A* to denote that they belong to Alice).

Alice transmits the domain parameters *q*, *p* and *g*, and her public key *KA*, to Bob.

In [6]:
# Bob
dh.validate_parameters(q, p, g)
dh.validate_pub_key(KA, q, p)

Having received the domain parameters *q*, *p* and *g*, and public key *KA* from Alice, Bob calls the same function as Alice to validate the domain parameters. Then he calls a different function `dh.validate_pub_key` to validate Alice's public key *KA*.

If neither function raises an exception, Bob knows that both the domain parameters and Alice's public key are valid, and can safely proceed with the protocol.

In [7]:
# Bob
kB, KB = dh.generate_keypair(q, p, g)

Using the same domain parameters *q*, *p* and *g* that Alice sent him, Bob now computes a key pair of his own, storing them in the variables *kB* and *KB*.

Then he transmits his public key *KB* to Alice.

In [8]:
# Alice
dh.validate_pub_key(KB, q, p)

On receiving Bob's public key *KB*, Alice must validate it in the same way Bob validated hers.

In [9]:
# Alice
kSessionA = dh.generate_session_key(KB, kA, q, p)

Next, Alice computes the single session key *kSessionA* that will be used to encrypt the message she will transmit to Bob.

To do this, she calls the function `dh.generate_session_key`, passing it Bob's public key *KB*, Alice's private key *kA*, and the public parameters *q* and *p*.

In [10]:
# Bob
kSessionB = dh.generate_session_key(KA, kB, q, p)

Bob calls the same function as Alice to compute a session key of his own, or *kSessionB*. Only in Bob's case, he passes the function Alice's public key *KA* and his private key *kB*, in addition to the parameters *q* and *p*.

Herein lies the essential property of DH: *KA* raised to the power of *kB* is equal to *KB* raised to the power *kA*, both modulo *p*, thus producing identical session keys *kSessionA* and *kSessionB*.

Where does this leave Eve? Having only seen *KA* and *KB*, Eve cannot from these values alone determine the value of the exponents used to compute them (i.e., she cannot solve the discrete logarithm problem, at least not in a reasonably short period of time). This is the *one&ndash;way* function that lies at the heart of DH.

As with all traditional ciphers, the key Alice uses to encrypt the message is the same one Bob uses to decrypt it. The difference here is that encryption is not predicated on a secure *exchange* of keys; rather, the keys are derived independently by Alice and Bob, and never transmitted over the insecure channel. Because of this, it is perhaps more apt to describe DH as a key *agreement* protocol rather than a key *exchange* protocol.

In [11]:
print(kSessionA == kSessionB)

True


If the essential property of DH holds, both keys&mdash;*kSessionA* and *kSessionB*&mdash;must be equivalent.

In [12]:
# Alice
mA = "Encrypt me!"

Next, Alice composes a confidential message for Bob, and stores it in the variable *mA*.

In [13]:
# Alice
mAC = sym.encrypt(kSessionA, mA)

She encrypts this message by calling the function `sym.encrypt`, passing it her session key *kSessionA* and the plaintext of her message *mA*. The function returns the ciphertext of *mA*, or *mAC*.

In [14]:
print(mAC)

53619526865502140145789225183468207442760176857237117786953246712923755200276


Here, we print the ciphertext *mAC*, which appears to Eve as an indecipherable string of numbers. Alice transmits the ciphertext *mAC* to Bob over the insecure channel.

In [15]:
# Bob
mB = sym.decrypt(kSessionB, mAC)

Bob, having received the ciphertext *mAC* from Alice, decrypts it by calling the function `sym.decrypt`, passing it his session key *kSessionB* and the ciphertext *mAC*. The result *mB* should be the plaintext of Alice's original message *mA*.

In [16]:
print(mB)

Encrypt me!


Printing *mB*, we see that Bob has successfully recovered the plaintext of Alice's message.

Finally, Eve, having observed the domain parameters *p*, *q* and *g*, and Alice's and Bob's public keys *KA* and *KB* (but not their private keys *kA* and *kB*), has no feasible way of deriving the session key *kSessionA* (or *kSessionB*). And without the session key, Eve cannot decrypt the message.